In [1]:
%run 0.1-data-import.ipynb

## 1.0 Early Exploratory Data Analysis

#### 1.0.1 Basic Data Checks

In [2]:
dfs = ['train', 'stores', 'oil']
unique_columns = [['store_nbr', 'family', 'date'], 
                  ['store_nbr'], 
                  ['date']]

for df_name, uni in zip(dfs, unique_columns):
    df = globals()[df_name]  # Access the DataFrame

    print(f'\033[1m{df_name}:\033[0m')
    
    print('\033[1mTypes\033[0m')
    print(df.dtypes)
    
    print('\033[1mShape\033[0m')
    print(df.shape)

    print('\033[1mMissing Values\033[0m')
    print(df.isnull().sum())
    
    print('\033[1mDuplicates\033[0m')
    print(df.duplicated(subset=uni, keep=False).sum())

    # Check for numeric columns
    if df.select_dtypes(include=[np.number]).empty:
        print('No numeric columns to describe')
    else:
        print('\033[1mNumeric Description\033[0m')
        print(df.describe().T)

    # Check for categorical columns
    if df.select_dtypes(include=['category']).empty:
        print('No categorical columns to describe')
    else:
        print('\033[1mCategoric Description\033[0m')
        print(df.describe(include=['category']).T)

    # Check for 'date' in unique columns and print min/max dates
    if 'date' in uni:
        print('\033[1mDate Range\033[0m')
        print(f"Min Date: {df['date'].min()}")
        print(f"Max Date: {df['date'].max()}")

    print()  # Print a blank line for readability

train:
Types
date           period[D]
store_nbr       category
family          category
sales            float32
onpromotion       uint32
dtype: object
Shape
(2950992, 5)
Missing Values
date           0
store_nbr      0
family         0
sales          0
onpromotion    0
dtype: int64
Duplicates
0
Numeric Description
                 count   mean     std  min  25%   50%    75%       max
sales       2950992.00 355.88 1098.83 0.00 0.00 10.00 194.03 124717.00
onpromotion 2950992.00   2.54   12.11 0.00 0.00  0.00   0.00    741.00
Categoric Description
             count unique         top   freq
store_nbr  2950992     54           1  54648
family     2950992     33  AUTOMOTIVE  89424
Date Range
Min Date: 2013-01-01
Max Date: 2017-07-18

stores:
Types
store_nbr    category
city         category
state        category
type         category
cluster      category
dtype: object
Shape
(54, 5)
Missing Values
store_nbr    0
city         0
state        0
type         0
cluster      0
dtype: int64
Dupl

#### 1.0.2 Preprocess

In [3]:
# Fill missing values in the 'dcoilwtico' column using backfill strategy
oil['dcoilwtico'] = oil['dcoilwtico'].fillna(method='backfill')

In [4]:
# Check the completeness of the train dataset
expected_dates = pd.period_range(start=train['date'].min(), end=train['date'].max())

missing_dates = expected_dates[~expected_dates.isin(train['date'])]

if len(missing_dates) == 0:
    print("The train dataset is complete. It includes all the required dates.")
else:
    print("The train dataset is incomplete. The following dates are missing:")
    print(missing_dates)

The train dataset is incomplete. The following dates are missing:
PeriodIndex(['2013-12-25', '2014-12-25', '2015-12-25', '2016-12-25'], dtype='period[D]')


<div class="alert alert-block alert-info">

The analysis will focus on the top 10 product groups. This targeted approach is intended to optimize the analysis by prioritizing product groups that have the most significant impact in terms of financials.

</div>

In [5]:
top_family = train.groupby('family')['sales'].sum().sort_values()

fig = px.bar(top_family.reset_index(), 
             x='sales', 
             y='family', 
             orientation='h', 
             log_x=True,  # Log scale for the x-axis
             title="Sales of Product Families")

fig.update_layout(xaxis_title="Average Sales (in Log scale)", 
                  yaxis_title="Product Family")

fig.show()

In [6]:
# Preprocess train data 
top_10_families = top_family.nlargest(10).index
train_filt = train[train['family'].isin(top_10_families)]

data = train_filt.merge(stores, on='store_nbr', how='left')
data = data.merge(oil, on='date', how='left')
data['dcoilwtico'] = data['dcoilwtico'].fillna(method='backfill')

comp_dir = Path('../data/processed/notebooks')
data.to_csv(
    comp_dir / 'train.csv',
    index=False
    )